In [1]:

import math
def InitialDataReducedMass(orbitalangle,r0):
    phi=orbitalangle
    radiusseparation=r0 #separation one hundred
    eccentricity=np.zeros(2)
    mass=np.ones(2)
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    masses=mass #*masssun (natural units)
    return phi, radiusseparation,eccentricity, masses
    

In [2]:
def getxyuveqreducedmass(initdat): #using reduced mass cooordinate system
    phi,radiusseparation,eccentricity, masses=initdatreducedmass
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    x0=np.zeros(2)
    y0=np.zeros(2)
    x0[1]=radiusseparation*np.cos(phi)*metersperAU
    y0[1]=radiusseparation*np.sin(phi)*metersperAU
    if phi==0:
        print("zero")
        x0[1]=radiusseparation
        y0[1]=0
    if phi==math.pi:
        print("pi")
        x0[1]=-radiusseparation
        y0[1]=0
    z0=np.zeros(2)
    print("x0", x0)
    print("y0", y0)
    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2) #*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    masstotal=np.sum(masses)
    reducedmass[0]=masstotal
    print(masses)
    for i in np.arange(1):
        j=(i+1)%2 #reverse masses
        reducedmass[i+1]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    metersperAU=1 #natural units
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        #rphys is r in the reduced mass system
        rphys[i]=radiusseparation #*metersperAU*reducedmass[i]/masses[i]
    x0[0]=0.0
    x0phys=rphys*x0/radiusseparation
    x0phys[0]=0.0
    y0phys=rphys*y0/radiusseparation
    z0phys=np.zeros(2)
    r0=np.sqrt(x0**2+y0**2)
    print(rphys)
    #G=1
    Gconstant=1
    F=(Gconstant*masstotal**2/(2*rphys)**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*masstotal/(rphys))
    print(v)
    print("r0", r0)
    ux0=-v*y0/radiusseparation #[1]
    uy0=v*x0/radiusseparation #[1]
        
    #ux0=-v*y0/r0
    #uy0=v*x0/r0 #initial data in y only
    #evolve in plane only
    #circular orbit
    uz0=np.zeros(2)
    
    a=Gconstant*masstotal/((2*rphys)**2)
    ax0=-a*x0/radiusseparation
    ay0=-a*y0/radiusseparation
    az0=np.zeros(2)
    
    
    return masstotal, reducedmass,x0phys,y0phys,z0phys, ux0, uy0,uz0, ax0, ay0,az0

In [4]:
import random,numpy as np
initdatreducedmass=InitialDataReducedMass(0,50)

In [11]:
xyuvaeqreducedmass=getxyuveqreducedmass(initdatreducedmass)
print(xyuvaeqreducedmass)
masstotal, reducedmass,x0phys,y0phys,z0phys, ux0, uy0,uz0, ax0, ay0,az0=xyuvaeqreducedmass

zero
x0 [ 0. 50.]
y0 [0. 0.]
[1. 1.]
[2.  0.5]
[50. 50.]
[0.0004 0.0004]
[0.2 0.2]
r0 [ 0. 50.]
(2.0, array([2. , 0.5]), array([ 0., 50.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([0. , 0.2]), array([0., 0.]), array([-0.    , -0.0002]), array([-0., -0.]), array([0., 0.]))


In [12]:
xvec=[x0phys,y0phys,z0phys,ux0,uy0,uz0]

In [13]:
def RK4implicit(h,t,xvec,f): #not a finite difference so no step in y
    k1= h*f(t,xvec)
    k2=h*f(t+h/2, xvec+k1/2)
    k3=h*f(t+h/2,xvec+k2/2)
    k4=h*f(t+h,xvec+k3)
    return t+h, xvec+1/6.*(k1+2.*k2+2.*k3+k4)

In [14]:
class OrbitDiffEq:
    def __init__(self,reducedmass,xvec,t0):
        self.reducedmass=reducedmass
        self.xveci=xvec
        self.ti=t0
    def dxidt(self,t,xvec):
        return xvec[3]
    def dyidt(self,t,xvec):
        return xvec[4]
    def dzidt(self,t,xvec):
        return xvec[5]
    def dvxidt(self,t,xvec):
        #return axi[m]
        axii=np.zeros(np.shape(self.axi))
        axii+=w
        rii=np.sqrt(xvec[0][:]**2+xvec[1][:]**2+xvec[2][:]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    axii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[0][j] - xvec[0][k])**2+(xvec[1][j]-xvec[1][k])**2+(xvec[2][j]-xvec[2][k])**2)**(1./2.)
                    axii[j]-=Gconstant*self.reducedmass[k]*(xvec[0][j]  - xvec[0][k])/rreljk**3
        self.axi=axii
        return axii
    def dvyidt(self,t,xvec):
        #return axi[m]
        ayii=np.zeros(np.shape(self.ayi))
        ayii+=w
        rii=np.sqrt(xvec[0][:]**2+xvec[1][:]**2+xvec[2][:]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    ayii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[0][j] - xvec[0][k])**2+(yvec[1][j]-xvec[1][k])**2+(xvec[2][j]-xvec[2][k])**2)**(1./2.)
                    ayii[j]-=Gconstant*self.reducedmass[k]*(xvec[1][j]  - xvec[1][k])/rreljk**3
        self.ayi=ayii
        return ayii
    def dvzidt(self,t,xvec):
        #return axi[m]
        azii=np.zeros(np.shape(self.azi))
        azii+=w
        rii=np.sqrt(xvec[0][:]**2+xvec[1][:]**2+xvec[2][:]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    azii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[0][j] - xvec[0][k])**2+(xvec[1][j]-vec[1][k])**2+(xvec[2][j]-xvec[2][k])**2)**(1./2.)
                    azii[j]-=Gconstant*self.reducedmass[k]*(xvec[2][j]  - xvec[2][k])/rreljk**3
        self.azi=azii
        return azii
    def updateINTERNAL(self,xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii,tii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
        self.ti=tii
        return self
    def update(self,xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii,tii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
        self.ti=tii
    def print2D(self):
        print(self.reducedmass,self.xi,self.yi,self.zi,self.vxi,self.vyi,self.vzi, self.axi,self.ayi,self.azi, self.ti)
        return self
    def list2D(self):
        return self.reducedmass,self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi,self.ti
    def timestepRK4ODE(self,step,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalx=RK4implicit(h,self.ti,self.xi,self.dxidt)
        xii = intvalx
        tnew,intvalvx=RK4implicit(h,self.ti,self.vxi,self.dvxidt)
        vxii=intvalvx
        tnew,intvaly=RK4implicit(h,self.ti,self.yi,self.dyidt)
        yii = intvaly
        tnew,intvalvy=RK4implicit(h,self.ti,self.vyi,self.dvyidt)
        vyii=intvalvy
        tnew,intvalz=RK4implicit(h,self.ti,self.zi,self.dzidt)
        zii = intvalz
        tnew,intvalvz=RK4implicit(h,self.ti,self.vzi,self.dvzidt)
        vzii=intvalvz
        axii=self.dvxidt(self.ti,np.zeros(np.shape(self.xi)))
        ayii=self.dvyidt(self.ti,np.zeros(np.shape(self.yi)))
        azii=self.dvzidt(self.ti,np.zeros(np.shape(self.zi)))
 
        #print(xii)
        self.updateINTERNAL(xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii,self.ti+dt)
        return reducedmass, xii,yii,zii,vxii,vyii,vzii,self.axi,self.ayi,self.azi,self.ti